In [1]:
from datetime import timedelta
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import plotly.express as px
from sodapy import Socrata
from keplergl import KeplerGl
import altair
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn import datasets
import ipywidgets
import requests
import seaborn as sns
import glob
import json
import os

In [2]:
dataset = pd.read_csv('merged_dataset.csv')
dataset

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895480,D438F1622839AC50,classic_bike,2022-12-06 15:43:38,2022-12-06 15:53:57,Dey St,JC065,Riverview Park,JC057,40.737828,-74.067083,40.744319,-74.043991,member
895481,747A63A8E782D171,electric_bike,2022-12-08 08:17:51,2022-12-08 08:23:33,9 St HBLR - Jackson St & 8 St,HB305,City Hall - Washington St & 1 St,HB105,40.747907,-74.038412,40.737360,-74.030970,casual
895482,AE090858CFDE6E82,electric_bike,2022-12-23 14:10:07,2022-12-23 14:14:18,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743140,-74.040041,40.737360,-74.030970,member
895483,B3CC8E70AF4E259C,classic_bike,2022-12-02 04:43:25,2022-12-02 04:46:55,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743135,-74.040080,40.737360,-74.030970,member


In [3]:
print(dataset.info())
print(dataset.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895485 entries, 0 to 895484
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             895485 non-null  object 
 1   rideable_type       895485 non-null  object 
 2   started_at          895485 non-null  object 
 3   ended_at            895485 non-null  object 
 4   start_station_name  895475 non-null  object 
 5   start_station_id    895475 non-null  object 
 6   end_station_name    892281 non-null  object 
 7   end_station_id      892281 non-null  object 
 8   start_lat           895485 non-null  float64
 9   start_lng           895485 non-null  float64
 10  end_lat             893515 non-null  float64
 11  end_lng             893515 non-null  float64
 12  member_casual       895485 non-null  object 
dtypes: float64(4), object(9)
memory usage: 88.8+ MB
None
ride_id                  0
rideable_type            0
started_at               0


In [4]:
dataset = dataset.dropna(how='any',axis=0)
print("Null values removed successfully.")
print(dataset.isnull().sum())
print(dataset.duplicated().any())

Null values removed successfully.
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64
False


In [5]:
pd.options.mode.chained_assignment = None

dataset["started_at"] = pd.to_datetime(dataset["started_at"])
dataset["ended_at"] = pd.to_datetime(dataset["ended_at"])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892281 entries, 0 to 895484
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             892281 non-null  object        
 1   rideable_type       892281 non-null  object        
 2   started_at          892281 non-null  datetime64[ns]
 3   ended_at            892281 non-null  datetime64[ns]
 4   start_station_name  892281 non-null  object        
 5   start_station_id    892281 non-null  object        
 6   end_station_name    892281 non-null  object        
 7   end_station_id      892281 non-null  object        
 8   start_lat           892281 non-null  float64       
 9   start_lng           892281 non-null  float64       
 10  end_lat             892281 non-null  float64       
 11  end_lng             892281 non-null  float64       
 12  member_casual       892281 non-null  object        
dtypes: datetime64[ns](2), float64

In [6]:
dataset["start_hour"] = dataset["started_at"].dt.hour
dataset["start_hour"].unique()
dataset['Date'] = dataset['started_at'].apply(lambda x: x.strftime('%Y-%m-%d'))

dataset["start_day_num"] = dataset["started_at"].dt.dayofweek
day_dict = {0:"Sunday", 1:"Monday", 2:"Tuesday", 3:"Wednesday", 4:"Thursday", 5:"Friday", 6:"Saturday"}
dataset["start_day_name"]= dataset["start_day_num"].apply(lambda y:day_dict[y])

dataset["day_of_year"] = dataset["started_at"].dt.day_of_year

dataset['start_month_num'] = dataset['started_at'].dt.month
dataset["start_month_num"].unique()

month_dict = {1:"January", 2:"February", 3:"March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}
dataset["start_month_name"]= dataset["start_month_num"].apply(lambda y:month_dict[y])
dataset["start_month_name"].unique()

dataset['Quarter'] = dataset['started_at'].apply(lambda x: int((int(x.strftime('%m')) - 1) / 3) + 1)

dataset['start_year'] = dataset['started_at'].dt.year
dataset["start_year"].unique()

dataset["trip_duration"] = dataset["ended_at"] - dataset["started_at"]
dataset["trip_duration"] = pd.to_numeric(dataset["trip_duration"])/6e+10

trip_mins = dataset[["started_at", "ended_at", "trip_duration"]]
trip_mins.nlargest(10, ["trip_duration"])

,started_at,ended_at,trip_duration
230426,2022-05-01 00:30:35,2022-05-20 20:12:05,28541.500000
366811,2022-07-10 14:05:49,2022-07-27 05:21:52,23956.050000
440050,2022-07-05 16:09:31,2022-07-18 17:19:15,18789.733333
279737,2022-06-24 15:39:26,2022-07-07 06:52:57,18193.516667
390934,2022-07-05 20:54:52,2022-07-17 09:53:51,16618.983333
370331,2022-07-04 18:08:36,2022-07-15 16:43:41,15755.083333
360382,2022-07-03 23:32:02,2022-07-14 14:02:10,15270.133333
611258,2022-09-10 22:47:29,2022-09-20 09:49:07,13621.633333
590466,2022-09-18 04:49:19,2022-09-26 11:59:41,11950.366667
466851,2022-08-25 08:47:42,2022-09-02 15:22:47,11915.083333


In [7]:
outliers_trip_duration = (dataset["trip_duration"] < 2) | (dataset["trip_duration"] > 500)
print(outliers_trip_duration.value_counts())
dataset.drop(dataset.index[outliers_trip_duration], inplace = True)
trip_mins = dataset[["started_at", "ended_at", "trip_duration"]]
trip_mins.nlargest(10, ["trip_duration"])

False    846875
True      45406
Name: trip_duration, dtype: int64


,started_at,ended_at,trip_duration
518707,2022-08-05 00:47:51,2022-08-05 09:07:46,499.916667
298755,2022-06-24 09:27:28,2022-06-24 17:46:49,499.350000
216587,2022-05-16 00:43:02,2022-05-16 09:01:41,498.650000
859440,2022-12-18 04:29:09,2022-12-18 12:47:22,498.216667
110902,2022-04-02 13:33:12,2022-04-02 21:51:19,498.116667
579195,2022-08-21 16:35:46,2022-08-22 00:53:41,497.916667
272147,2022-06-09 21:49:36,2022-06-10 06:07:16,497.666667
582912,2022-09-02 00:49:32,2022-09-02 09:07:04,497.533333
271430,2022-06-23 04:50:48,2022-06-23 13:08:17,497.483333
632563,2022-09-02 00:49:49,2022-09-02 09:07:04,497.250000


In [8]:
map = KeplerGl(height=600,width=800)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [9]:
map.add_data(data=dataset,name='trip locations')

In [11]:
dataset.to_csv('dataset_mod.csv', index = False)